# Test statistics

We define two test statistics. 

The first flattens all normalized compatibility scores across all 11 379 metrical positions, and takes the mean.

Let $T_{pos}$ denote this statistic:

$$ T_{pos} = \frac{1}{11 379} \sum_{pos=1}^{11 379} \bar{x}_{pos}.$$

To avoid longer songs weighing more, we also define a second test statistic that first averages the normalized compatibility scores within each song, and then takes the mean again across the 40 songs. 

Let $T_{song}$ denote this statistic:

$$ T_{song} = \frac{1}{40} \sum_{s=1}^{40} \bar{x}_s.$$

We are going to compute these test statistics for the Pindar corpus, and for 10 000 random baselines, and then perform a one-sided permutation test to see if the observed statistics are significantly higher than the baselines.


## Observed distribution (Pindar)

In [11]:
from pathlib import Path
ROOT = Path.cwd().parent

Let's start with $T_{pos}$ for the Pindar corpus. 

In [12]:
import pickle

from stats_comp import compatibility_corpus, compatibility_canticum, compatibility_ratios_to_stats

corpus_path = ROOT / 'data/compiled/triads'
corpus_path_strophes = ROOT / 'data/compiled/strophes/'
corpus_path_epodes = ROOT / 'data/compiled/epodes/'

corpus_score_pickle_path = ROOT / 'data/cache/all_comp_ratios.pkl'
corpus_score_pickle_path_strophes = ROOT / 'data/cache/all_comp_ratios_strophes.pkl'
corpus_score_pickle_path_epodes = ROOT / 'data/cache/all_comp_ratios_epodes.pkl'

if corpus_score_pickle_path.exists():
    with open(corpus_score_pickle_path, 'rb') as f:
        all_corpus_comp_scores = pickle.load(f)
else:
    print('Calculating all_corpus_comp_scores...')
    all_corpus_comp_scores = compatibility_corpus(corpus_path)
    with open(corpus_score_pickle_path, 'wb') as f:
        pickle.dump(all_corpus_comp_scores, f)

if corpus_score_pickle_path_strophes.exists():
    with open(corpus_score_pickle_path_strophes, 'rb') as f:
        all_corpus_comp_scores_strophes = pickle.load(f)
else:
    print('Calculating all_corpus_comp_scores_strophes...')
    all_corpus_comp_scores_strophes = compatibility_corpus(corpus_path_strophes)
    with open(corpus_score_pickle_path_strophes, 'wb') as f:
        pickle.dump(all_corpus_comp_scores_strophes, f)

if corpus_score_pickle_path_epodes.exists():
    with open(corpus_score_pickle_path_epodes, 'rb') as f:
        all_corpus_comp_scores_epodes = pickle.load(f)
else:
    print('Calculating all_corpus_comp_scores_epodes...')
    all_corpus_comp_scores_epodes = compatibility_corpus(corpus_path_epodes)
    with open(corpus_score_pickle_path_epodes, 'wb') as f:
        pickle.dump(all_corpus_comp_scores_epodes, f)

all_corpus_comp_scores_ba = compatibility_canticum(ROOT / 'data/compiled/extra/ba05.xml', 'ba05')

def count_leaves(obj):
    """Count all leaf (non-list) elements in a nested structure."""
    if isinstance(obj, list):
        return sum(count_leaves(item) for item in obj)
    else:
        return 1

# Calculate T_obs for each category
T_obs_pos_triads = compatibility_ratios_to_stats(all_corpus_comp_scores)
T_obs_triads_ba = compatibility_ratios_to_stats(all_corpus_comp_scores_ba) # For Bacchylides T_pos = T_song

T_obs_pos_strophes = compatibility_ratios_to_stats(all_corpus_comp_scores_strophes)
T_obs_pos_epodes = compatibility_ratios_to_stats(all_corpus_comp_scores_epodes)

# Count total number of variables
total_triads = sum(count_leaves(lst) for lst in all_corpus_comp_scores)
total_triads_ba = sum(count_leaves(lst) for lst in all_corpus_comp_scores_ba)
total_strophes = sum(count_leaves(lst) for lst in all_corpus_comp_scores_strophes)
total_epodes = sum(count_leaves(lst) for lst in all_corpus_comp_scores_epodes)

print ('--------------------------------------------')
print(f'\033[1;33mTRIADS\033[0m\n')
print(f'\033[33mPindar\033[0m: T_pos: \033[1;32m{T_obs_pos_triads:.3f}\033[0m')
print(f'Number of variables: {total_triads}')

print(f'\033[33mBacchylides\033[0m (Ba. 5) T (T_pos = T_song): \033[1;32m{T_obs_triads_ba:.3f}\033[0m')
print(f'Number of variables: {total_triads_ba}')
print ('--------------------------------------------')

print ('\n--------------------------------------------')
print(f'\033[1;33mSTROPHES\033[0m\n')
print(f'\033[33mPindar\033[0m: T_pos: \033[1;32m{T_obs_pos_strophes:.3f}\033[0m')
print(f'Number of variables: {total_strophes}')
print ('--------------------------------------------')

print ('\n--------------------------------------------')
print(f'\033[1;33mEPODES\033[0m\n')
print(f'\033[33mPindar\033[0m: T_pos: \033[1;32m{T_obs_pos_epodes:.3f}\033[0m')
print(f'Number of variables: {total_epodes}')
print ('--------------------------------------------')


--------------------------------------------
TRIADS

Pindar: T_pos: 0.412
Number of variables: 7361
Bacchylides (Ba. 5) T (T_pos = T_song): 0.371
Number of variables: 221
--------------------------------------------

--------------------------------------------
STROPHES

Pindar: T_pos: 0.451
Number of variables: 3543
--------------------------------------------

--------------------------------------------
EPODES

Pindar: T_pos: 0.408
Number of variables: 1252
--------------------------------------------


Now let's also do $T_{song}$ for all three categories (slightly more work to do here, since we need to average within songs first, and 
- 40 songs respond *simpliciter*,
- only 32 songs are truly triadic, i.e. have responding strophe-antistrophe pairs and epodes),
- 1 song (py07) has only one triad strophe element, hence there is *only* strophic-antistrophic responsion and neither triadic nor epodic responsion 

In [10]:
import xml.etree.ElementTree as ET

FOLDERS = {
    "triads": ROOT / "data" / "compiled" / "triads",
    "strophes": ROOT / "data" / "compiled" / "strophes",
    "epodes": ROOT / "data" / "compiled" / "epodes",
}

def collect_ids(folder: Path) -> set[str]:
    ids: set[str] = set()
    for xml_path in folder.glob("*.xml"):
        print(xml_path)
        try:
            tree = ET.parse(xml_path)
            for elem in tree.iter():
                resp_id = elem.attrib.get("responsion")
                if resp_id:
                    ids.add(resp_id)
        except Exception as exc:
            print(f"Warning: failed to parse {xml_path}: {exc}")
    return ids

def main() -> None:
    results: dict[str, set[str]] = {}
    for name, folder in FOLDERS.items():
        ids = collect_ids(folder)
        results[name] = ids
        print(f"{name}: {len(ids)} unique responsion id values")
        if ids:
            print("IDs:", ", ".join(sorted(ids)))
        print()

    tri, stro, epo = results["triads"], results["strophes"], results["epodes"]
    print("Unique to triads:", ", ".join(sorted(tri - stro - epo)) or "None")
    print("Unique to strophes:", ", ".join(sorted(stro - tri - epo)) or "None")
    print("Unique to epodes:", ", ".join(sorted(epo - tri - stro)) or "None")
    print("\nDone.")

main()

/Users/albin/git/responsio-accentuum/data/compiled/triads/ht_nemeans_triads.xml
/Users/albin/git/responsio-accentuum/data/compiled/triads/ht_isthmians_triads.xml
/Users/albin/git/responsio-accentuum/data/compiled/triads/ht_pythians_triads.xml
/Users/albin/git/responsio-accentuum/data/compiled/triads/ht_olympians_triads.xml
triads: 40 unique responsion id values
IDs: is01, is02, is04, is05, is06, is07, is08, ne01, ne02, ne03, ne04, ne05, ne06, ne07, ne08, ne09, ne10, ne11, ol01, ol02, ol03, ol05, ol06, ol07, ol08, ol09, ol10, ol13, ol14, py01, py02, py03, py04, py05, py06, py08, py09, py10, py11, py12

/Users/albin/git/responsio-accentuum/data/compiled/strophes/ht_olympians_strophes.xml
/Users/albin/git/responsio-accentuum/data/compiled/strophes/ht_isthmians_strophes.xml
/Users/albin/git/responsio-accentuum/data/compiled/strophes/ht_nemeans_strophes.xml
/Users/albin/git/responsio-accentuum/data/compiled/strophes/ht_pythians_strophes.xml
strophes: 37 unique responsion id values
IDs: is01

In [13]:
from stats_comp import compatibility_canticum, compatibility_ratios_to_stats
from utils.utils import victory_odes, canticum_with_at_least_two_strophes

song_scores_triads = {}
song_scores_strophes = {}
song_scores_epodes = {}

responding_unit = ["triads", "strophes", "epodes"]
for unit in responding_unit:
    in_folder_odes = ROOT / f"data/compiled/{unit}/"
    for ode in sorted(victory_odes):
        if ode[:2] == 'py':
            ode_path = in_folder_odes / f"ht_pythians_{unit}.xml"
        elif ode[:2] == 'ne':
            ode_path = in_folder_odes / f"ht_nemeans_{unit}.xml"
        elif ode[:2] == 'ol':
            ode_path = in_folder_odes / f"ht_olympians_{unit}.xml"
        elif ode[:2] == 'is':
            ode_path = in_folder_odes / f"ht_isthmians_{unit}.xml"
        
        # Check if the canticum exists and has at least 2 strophes
        if not canticum_with_at_least_two_strophes(ode_path, ode):
            continue
        
        try:
            ode_ratios = compatibility_canticum(ode_path, ode)
            ode_comp = compatibility_ratios_to_stats(ode_ratios)

            if unit == "triads":
                song_scores_triads[ode] = ode_comp
            elif unit == "strophes":
                song_scores_strophes[ode] = ode_comp
            elif unit == "epodes":
                song_scores_epodes[ode] = ode_comp
        except Exception as e:
            print(f"Warning: Could not process {ode} in {unit}: {e}")
            continue

# Calculate T_obs_song
T_obs_song_triads = compatibility_ratios_to_stats(list(song_scores_triads.values()))
T_obs_song_strophes = compatibility_ratios_to_stats(list(song_scores_strophes.values()))
T_obs_song_epodes = compatibility_ratios_to_stats(list(song_scores_epodes.values()))

print(f"T_obs_song_triads: {T_obs_song_triads:.3f}")
print(f"T_obs_song_strophes: {T_obs_song_strophes:.3f}")
print(f"T_obs_song_epodes: {T_obs_song_epodes:.3f}")

# Print how many songs have only triads
print("\nAntistrophic songs (no antistrophes or epodes):")
count = 0
for ode in sorted(victory_odes):
    if ode in song_scores_triads and ode not in song_scores_strophes:
        print(f"- {ode}")
        count += 1
print(f"Total: {count}")

for ode in sorted(victory_odes):

    print('--------------------------------------------')
    print(f'\033[1;32m{ode}\033[0m')
    if ode in song_scores_triads:
        print(f'Ode comp: \033[1;32m{song_scores_triads[ode]:.3f}\033[0m (triads)')
    else:
        print(f'Ode comp: N/A (triads - not enough strophes)')
    
    if ode in song_scores_strophes:
        print(f'Ode comp: \033[1;32m{song_scores_strophes[ode]:.3f}\033[0m (strophes)')
    else:
        print(f'Ode comp: N/A (strophes - not enough strophes)')
    
    if ode in song_scores_epodes:
        print(f'Ode comp: \033[1;32m{song_scores_epodes[ode]:.3f}\033[0m (epodes)')
    else:
        print(f'Ode comp: N/A (epodes - not enough strophe elements)')
    print('--------------------------------------------')

T_obs_song_triads: 0.415
T_obs_song_strophes: 0.437
T_obs_song_epodes: 0.412

Antistrophic songs (no antistrophes or epodes):
- is07
- is08
- ne02
- ne04
- ne09
- ol14
- py06
- py12
Total: 8
--------------------------------------------
is01
Ode comp: 0.500 (triads)
Ode comp: 0.431 (strophes)
Ode comp: 0.516 (epodes)
--------------------------------------------
--------------------------------------------
is02
Ode comp: 0.447 (triads)
Ode comp: 0.509 (strophes)
Ode comp: 0.507 (epodes)
--------------------------------------------
--------------------------------------------
is04
Ode comp: 0.408 (triads)
Ode comp: 0.449 (strophes)
Ode comp: 0.431 (epodes)
--------------------------------------------
--------------------------------------------
is05
Ode comp: 0.351 (triads)
Ode comp: 0.499 (strophes)
Ode comp: 0.359 (epodes)
--------------------------------------------
--------------------------------------------
is06
Ode comp: 0.402 (triads)
Ode comp: 0.498 (strophes)
Ode comp: 0.400 (ep

## Expected distributions (baselines)

In [14]:
from baseline import one_t_prose

(T_pos_prose, T_song_prose) = one_t_prose()

print("T_pos_prose:", f"{T_pos_prose:.4f}")
print("T_song_prose:", f"{T_song_prose:.4f}")

T_pos_prose: 0.4053
T_song_prose: 0.4068


And for the lyric! The lyric has more moving parts and needs to be tested separately, since the randomization procedure is different (see `baseline.py` for details).

In [15]:
from baseline import one_t_lyric

(T_pos_lyric, T_song_lyric) = one_t_lyric()

print("T_pos_lyric:", f"{T_pos_lyric:.4f}")
print("T_song_lyric:", f"{T_song_lyric:.4f}")

T_pos_lyric: 0.3956
T_song_lyric: 0.3947


As expected, *T_pos is slightly lower than T_song* (since longer songs tend to more strictions on responsion) and *the lyric baselines are slightly lower than the prose* (because lyric is also more constrained than prose).

Now let's calculate lists of test statstics for n randomizations of the corpus.

In [ ]:
import pickle

from baseline import test_statistics

randomizations = 100
workers = 8
chunk_size = 10
T_pos_prose_list, T_song_prose_list, T_pos_lyric_list, T_song_lyric_list, lyric_stats_summary = test_statistics(
    randomizations,
    workers=workers,
    chunk_size=chunk_size,
    include_lyric_stats=True,
)

print("\nFirst sample in each test statistic series:\n")
print(f"T_pos_prose_list: \033[1;32m{T_pos_prose_list[0]:.3f}\033[0m")
print(f"T_song_prose_list: \033[1;32m{T_song_prose_list[0]:.3f}\033[0m")
print(f"T_pos_lyric_list: \033[1;32m{T_pos_lyric_list[0]:.3f}\033[0m")
print(f"T_song_lyric_list: \033[1;32m{T_song_lyric_list[0]:.3f}\033[0m")

if lyric_stats_summary:
    print("\nLyric baseline composition (aggregated):")
    for k, v in lyric_stats_summary.items():
        print(f"  {k}: {v}")

pickle_output = ROOT / "data/cache/test_statistics.pkl"
with open(pickle_output, "wb") as f:
    pickle.dump((T_pos_prose_list, T_song_prose_list, T_pos_lyric_list, T_song_lyric_list, lyric_stats_summary), f)

Test statistics (computing):  12%|█▎        | 10/80 [01:21<09:29,  8.14s/it]


Saved chunk 0_10 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_0_10.pkl

Saved chunk 10_20 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_10_20.pkl

Saved chunk 20_30 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_20_30.pkl

Saved chunk 30_40 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_30_40.pkl

Saved chunk 40_50 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_40_50.pkl

Saved chunk 50_60 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_50_60.pkl


Test statistics (computing): 100%|██████████| 80/80 [01:22<00:00,  1.03s/it]


Saved chunk 60_70 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_60_70.pkl

Saved chunk 70_80 to /Users/albin/git/responsio-accentuum/data/cache/test_statistics_chunks/chunk_70_80.pkl

Completed: 0 chunks from cache, 8 chunks computed (80 total iterations)

First sample in each test statistic series:

T_pos_prose_list: 0.405
T_song_prose_list: 0.407
T_pos_lyric_list: 0.396
T_song_lyric_list: 0.395

Lyric baseline composition (aggregated):
  total_lines: 269760
  pindar_lines: 269760
  external_lines: 0
  unaltered_lines: 257345
  trimmed_lines: 12415
  padded_lines: 0
  paired_fallbacks: 2080


In [14]:
import pickle 

pickle_input = ROOT / "data/cache/test_statistics.pkl"

with open(pickle_input, "rb") as f:
    T_pos_prose_list, T_song_prose_list, T_pos_lyric_list, T_song_lyric_list = pickle.load(f)

# Print first 5 values of each as .3f list to verify
print("T_pos_prose_list:", [f"{x:.3f}" for x in T_pos_prose_list[:5]])
print("T_song_prose_list:", [f"{x:.3f}" for x in T_song_prose_list[:5]])
print("T_pos_lyric_list:", [f"{x:.3f}" for x in T_pos_lyric_list[:5]])
print("T_song_lyric_list:", [f"{x:.3f}" for x in T_song_lyric_list[:5]])

T_pos_prose_list: ['0.405', '0.397', '0.397', '0.408', '0.403']
T_song_prose_list: ['0.407', '0.399', '0.399', '0.411', '0.405']
T_pos_lyric_list: ['0.396', '0.401', '0.399', '0.406', '0.392']
T_song_lyric_list: ['0.395', '0.402', '0.399', '0.407', '0.393']


We try to plot the results in a histogram, but the distribution is not very smooth, so we also plot the means of the baseline distributions as vertical lines.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_permutation_histogram(
    T_null,
    T_obs,
    title,
    xlabel,
    bins=40,
    color="lightgray"
):
    plt.figure(figsize=(8, 5))

    plt.hist(
        T_null,
        bins=bins,
        color=color,
        edgecolor="white"
    )

    plt.axvline(
        T_obs,
        color="black",
        linewidth=3,
        label="Observed value (Pindar)"
    )

    plt.xlabel(xlabel)
    plt.ylabel("Number of simulated corpora")
    plt.title(title)
    plt.legend(frameon=False)

    plt.tight_layout()
    plt.show()


# === MAIN FIGURE ===
plot_permutation_histogram(
    T_null=T_song_lyric_list,
    T_obs=T_obs,
    title="Permutation distribution under lyric baseline",
    xlabel="Mean melodic compatibility (song-level)"
)

# Significance test

We perform a one-sided permutation test, because the hypothesis is directional.

Let $T_{\text{obs}} =$ observed statistic (Pindar)
and $\{T_1^{(0)}, \dots, T_{10000}^{(0)}\} =$ baseline values

Compute, for each list in T_pos_prose_list, T_song_prose_list, T_pos_lyric_list, T_song_lyric_list:

$$p = \frac{1 + \#\{T_i^{(0)} \ge T_{\text{obs}}\}}{1 + 10000}$$

$$z_{\text{emp}} = \frac{T_{\text{obs}} - \mu_0}{\sigma_0}$$

## Histograms 

It can be interesting to plot the observed distribution and the means of the two baseline distributions as histrograms with the bins of normalized compatibility scores. 